In [ ]:
# 가상환경 생성 (Python 3.10 또는 3.11)

# py -3.11 -m venv agent-env <- terminal에서 실행, venv 생성
# agent-env\Scripts\activate <- terminal에서 실행, venv 활성화

In [28]:
%pip install -r requirements.txt

Note: you may need to restart the kernel to use updated packages.


In [ ]:
# %pip install regex

Note: you may need to restart the kernel to use updated packages.


In [29]:
# from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

from langchain.agents import initialize_agent, Tool, AgentType
# from langchain.utilities import SerpAPIWrapper
from langchain_community.tools import DuckDuckGoSearchRun
from langchain.tools import Tool
# from langchain.llms import HuggingFacePipeline, HuggingFaceEndpoint
# from langchain.chat_models import ChatOpenAI
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
# from langchain.tools.tavily_search import TavilySearchResults

# from serpapi import GoogleSearch

# from huggingface_hub import login, HfApi

from dotenv import load_dotenv
import json
from jinja2 import Template
import os
import re
import json
import regex
# from datetime import datetime, timedelta

In [30]:
load_dotenv()
openai_api_key = os.getenv("OPENAI_API_KEY")

In [31]:
JSON_FILE_PATH = "C:\\Users\\pje03\\Documents\\pja_MLOps\\Task_recommend\\use_openai\\data\\final_normalized_dummy.json"
RCMD_PROMPT_PATH = "C:\\Users\\pje03\\Documents\\pja_MLOps\\Task_recommend\\prompts\\agent_prompt.md"

In [32]:
# Read prompt
with open(RCMD_PROMPT_PATH, "r", encoding="utf-8", errors="replace") as f:
    RCMD_PROMPT = f.read()
    
template = Template(RCMD_PROMPT)

# Read data
with open(JSON_FILE_PATH, 'r', encoding='utf-8') as f:
    data = json.load(f)
    print(type(data))  # test

<class 'list'>


In [34]:
WORKSPACE_ID = 0

def extract_workspace_view(id):
    if not isinstance(data, list):
        print("** data type is not list **")
    
    workspace_data = next((ws for ws in data if ws.get("workspaceId") == id), None)

    ##### test #####
    for ws in data:
        print(f"workspaceId: {ws.get('workspaceId')}, type: {type(ws.get('workspaceId'))}")

    if not workspace_data:
        raise ValueError(f"** workspaceId={id} does NOT exist **")
    
    return json.dumps(workspace_data, indent=4, ensure_ascii=False)

workspace_data = extract_workspace_view(WORKSPACE_ID)
print(workspace_data)
print(type(workspace_data))

workspaceId: 0, type: <class 'int'>
workspaceId: 1, type: <class 'int'>
workspaceId: 2, type: <class 'int'>
workspaceId: 3, type: <class 'int'>
workspaceId: 4, type: <class 'int'>
workspaceId: 5, type: <class 'int'>
workspaceId: 6, type: <class 'int'>
workspaceId: 7, type: <class 'int'>
workspaceId: 8, type: <class 'int'>
workspaceId: 9, type: <class 'int'>
workspaceId: 10, type: <class 'int'>
workspaceId: 11, type: <class 'int'>
workspaceId: 12, type: <class 'int'>
workspaceId: 13, type: <class 'int'>
workspaceId: 14, type: <class 'int'>
workspaceId: 15, type: <class 'int'>
workspaceId: 16, type: <class 'int'>
workspaceId: 17, type: <class 'int'>
workspaceId: 18, type: <class 'int'>
workspaceId: 19, type: <class 'int'>
workspaceId: 20, type: <class 'int'>
workspaceId: 21, type: <class 'int'>
workspaceId: 22, type: <class 'int'>
workspaceId: 23, type: <class 'int'>
workspaceId: 24, type: <class 'int'>
workspaceId: 25, type: <class 'int'>
workspaceId: 26, type: <class 'int'>
workspaceId

In [45]:
formatted_input = template.render(workspace_data=workspace_data)
print(formatted_input)

Question: 아래의 JSON 작업 흐름을 바탕으로 각 feature에 대해 이어질 작업 3개씩 추천해주세요. 추천 작업은 feature의 actions 배열에 포함되어야 하며, name, startDate, endDate, importance가 반드시 포함되어야 합니다. 결과는 반드시 JSON 형식으로 출력하고, 추가 설명 없이 JSON만 출력해야 합니다. 출력 예시는 아래 형식을 따릅니다:

{
  "workspaceId": "작업 중인 workspaceId",
  "recommendedCategories": [
    {
      "name": "카테고리 이름",
      "startDate": "YYYY-MM-DDTHH:MM:SS",
      "endDate": "YYYY-MM-DDTHH:MM:SS",
      "importance": 정수,
      "features": [
        {
          "name": "기능 이름",
          "startDate": "YYYY-MM-DDTHH:MM:SS",
          "endDate": "YYYY-MM-DDTHH:MM:SS",
          "importance": 정수,
          "actions": [
            {
              "name": "추천 action",
              "startDate": "YYYY-MM-DDTHH:MM:SS",
              "endDate": "YYYY-MM-DDTHH:MM:SS",
              "importance": 정수
            },
            {
              "name": "추천 action",
              "startDate": "YYYY-MM-DDTHH:MM:SS",
              "endDate": "YYYY-MM-DDTHH:MM:SS",
              "importance": 정수


In [46]:
llm = ChatOpenAI(
    model="gpt-4o",
    temperature=0.3,
    max_tokens=1024,
    openai_api_key=openai_api_key
)

In [47]:
prompt = PromptTemplate.from_template("""
다음은 '{featureName}' 기능의 기존 작업들입니다:

{existingActions}

이 작업의 다음 단계로서, 자연스럽게 이어질 수 있는 작업 3개를 추천하세요.
각 작업은 다음 항목을 포함해야 합니다:

- name: 구체적 작업명
- startDate: YYYY-MM-DDTHH:MM:SS 형식
- endDate: YYYY-MM-DDTHH:MM:SS 형식
- importance: 1~5 정수

형식은 반드시 JSON 배열 형태로 출력하세요.
예시:
```json
[
  {
    "name": "...",
    "startDate": "...",
    "endDate": "...",
    "importance": 3
  },
  ...
]
```
""")

# LLM 연결
llm = ChatOpenAI(temperature=0.4, model="gpt-4o-mini")
llm_chain = LLMChain(prompt=prompt, llm=llm)

# LLM으로 작업 추천 받는 함수
def generate_next_actions(input_dict: dict) -> str:
    return llm_chain.run(input_dict)

# 전체 JSON을 받아 feature별 추천 작업을 생성하는 메인 함수
def parse_and_generate_all(input_str: str) -> str:
    print("------ input_str BLOCK ------")
    print(input_str)
    print("---------- end line ----------")
    pattern = r'({\s*"workspaceId".*})'
    match = re.search(pattern, input_str, re.DOTALL)

    if not match:
        raise ValueError("** Can not fine JSON blocks **")

    json_block = match.group(0)
    data = json.loads(json_block)
    # data = json.loads(input_str)
    print("------ JSON BLOCK ------")
    print(json_block)
    print("------- end line -------")

    for category in data.get("recommendedCategories", []):
        for feature in category.get("features", []):
            llm_input = {
                "featureName": feature["name"],
                "existingActions": [a["name"] for a in feature.get("actions", [])]
            }

            try:
                generated_json_str = generate_next_actions(llm_input)
                generated_actions = json.loads(generated_json_str)
                feature["actions"].extend(generated_actions)
            except Exception as e:
                feature["actions"].append({
                    "name": f"작업 생성 실패: {e}",
                    "startDate": "2025-01-01T00:00:00",
                    "endDate": "2025-01-01T23:59:59",
                    "importance": 1
                })

    return json.dumps(data, ensure_ascii=False, indent=2)


In [48]:
parse_and_generate_all(workspace_data)

------ input_str BLOCK ------
{
    "workspaceId": 0,
    "recommendedCategories": [
        {
            "name": "스터디 그룹 관리 기능",
            "startDate": "2025-07-01T00:00:00",
            "endDate": "2025-07-20T23:59:59",
            "importance": 5,
            "features": [
                {
                    "name": "스터디 그룹 생성",
                    "startDate": "2025-07-02T00:00:00",
                    "endDate": "2025-07-05T23:59:59",
                    "importance": 4,
                    "actions": [
                        {
                            "name": "그룹 이름 및 설명 입력 폼 구현",
                            "startDate": "2025-07-02T00:00:00",
                            "endDate": "2025-07-03T23:59:59",
                            "importance": 3
                        },
                        {
                            "name": "그룹 생성 시 사용자 알림 기능 추가",
                            "startDate": "2025-07-03T00:00:00",
                            "endDate": "2025-07-04

'{\n  "workspaceId": 0,\n  "recommendedCategories": [\n    {\n      "name": "스터디 그룹 관리 기능",\n      "startDate": "2025-07-01T00:00:00",\n      "endDate": "2025-07-20T23:59:59",\n      "importance": 5,\n      "features": [\n        {\n          "name": "스터디 그룹 생성",\n          "startDate": "2025-07-02T00:00:00",\n          "endDate": "2025-07-05T23:59:59",\n          "importance": 4,\n          "actions": [\n            {\n              "name": "그룹 이름 및 설명 입력 폼 구현",\n              "startDate": "2025-07-02T00:00:00",\n              "endDate": "2025-07-03T23:59:59",\n              "importance": 3\n            },\n            {\n              "name": "그룹 생성 시 사용자 알림 기능 추가",\n              "startDate": "2025-07-03T00:00:00",\n              "endDate": "2025-07-04T23:59:59",\n              "importance": 2\n            },\n            {\n              "name": "그룹 생성 후 데이터베이스 저장 로직 구현",\n              "startDate": "2025-07-04T00:00:00",\n              "endDate": "2025-07-05T23:59:59",\n          

In [49]:
# 웹 검색 툴 정의
search = DuckDuckGoSearchRun()

parse_tool = Tool(
    name="ParseWorkflow",
    func=parse_and_generate_all,
    description="JSON 작업 흐름에 대해 feature별로 다음 추천 작업 3개를 추가합니다."
)

final_tool = Tool(
    name="Final Answer",
    func=lambda x: x,  # 결과를 그대로 반환
    description="최종 JSON 결과를 반환합니다."
)

In [50]:
# Agent
custom_prompt = formatted_input

# agent = initialize_agent(
#     tools=[parse_tool, final_tool],
#     llm=llm,
#     agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
#     # agent="zero-shot-react-description",
#     verbose=True,
#     handle_parsing_errors=True,
#     prompt=formatted_input
# )

agent_executor = initialize_agent(
    tools=[parse_tool, final_tool],
    llm=llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
    handle_parsing_errors=True,
    prompt=formatted_input
)

In [51]:
prompt_filled = RCMD_PROMPT.replace("{input}", json.dumps(workspace_data, ensure_ascii=False))

# response = agent.run({"input": prompt_filled})
response = agent_executor.invoke({"input": prompt_filled})
print(response)



> Entering new AgentExecutor chain...
{
  "workspaceId": 0,
  "recommendedCategories": [
    {
      "name": "스터디 그룹 관리 기능",
      "startDate": "2025-07-01T00:00:00",
      "endDate": "2025-07-20T23:59:59",
      "importance": 5,
      "features": [
        {
          "name": "스터디 그룹 생성",
          "startDate": "2025-07-02T00:00:00",
          "endDate": "2025-07-05T23:59:59",
          "importance": 4,
          "actions": [
            {
              "name": "그룹 이름 및 설명 입력 폼 구현",
              "startDate": "2025-07-02T00:00:00",
              "endDate": "2025-07-03T23:59:59",
              "importance": 3
            },
            {
              "name": "그룹 생성 시 사용자 알림 기능 추가",
              "startDate": "2025-07-03T00:00:00",
              "endDate": "2025-07-04T23:59:59",
              "importance": 2
            },
            {
              "name": "그룹 생성 후 데이터베이스 저장 로직 구현",
              "startDate": "2025-07-04T00:00:00",
              "endDate": "2025-07-05T23:59:59",
  

In [53]:
response

{'input': 'Question: 아래의 JSON 작업 흐름을 바탕으로 각 feature에 대해 이어질 작업 3개씩 추천해주세요. 추천 작업은 feature의 actions 배열에 포함되어야 하며, name, startDate, endDate, importance가 반드시 포함되어야 합니다. 결과는 반드시 JSON 형식으로 출력하고, 추가 설명 없이 JSON만 출력해야 합니다. 출력 예시는 아래 형식을 따릅니다:\n\n{\n  "workspaceId": "작업 중인 workspaceId",\n  "recommendedCategories": [\n    {\n      "name": "카테고리 이름",\n      "startDate": "YYYY-MM-DDTHH:MM:SS",\n      "endDate": "YYYY-MM-DDTHH:MM:SS",\n      "importance": 정수,\n      "features": [\n        {\n          "name": "기능 이름",\n          "startDate": "YYYY-MM-DDTHH:MM:SS",\n          "endDate": "YYYY-MM-DDTHH:MM:SS",\n          "importance": 정수,\n          "actions": [\n            {\n              "name": "추천 action",\n              "startDate": "YYYY-MM-DDTHH:MM:SS",\n              "endDate": "YYYY-MM-DDTHH:MM:SS",\n              "importance": 정수\n            },\n            {\n              "name": "추천 action",\n              "startDate": "YYYY-MM-DDTHH:MM:SS",\n              "endDate": "YYYY-MM-DDTHH:MM

In [15]:
print(type(prompt_filled))
print(prompt_filled)

<class 'str'>
Question: 아래의 JSON 작업 흐름을 바탕으로 각 feature에 대해 이어질 작업 3개씩 추천해주세요. 추천 작업은 feature의 actions 배열에 포함되어야 하며, name, startDate, endDate, importance가 반드시 포함되어야 합니다. 결과는 반드시 JSON 형식으로 출력하고, 추가 설명 없이 JSON만 출력해야 합니다. 출력 예시는 아래 형식을 따릅니다:

{
  "workspaceId": "작업 중인 workspaceId",
  "recommendedCategories": [
    {
      "name": "카테고리 이름",
      "startDate": "YYYY-MM-DDTHH:MM:SS",
      "endDate": "YYYY-MM-DDTHH:MM:SS",
      "importance": 정수,
      "features": [
        {
          "name": "기능 이름",
          "startDate": "YYYY-MM-DDTHH:MM:SS",
          "endDate": "YYYY-MM-DDTHH:MM:SS",
          "importance": 정수,
          "actions": [
            {
              "name": "추천 action",
              "startDate": "YYYY-MM-DDTHH:MM:SS",
              "endDate": "YYYY-MM-DDTHH:MM:SS",
              "importance": 정수
            },
            {
              "name": "추천 action",
              "startDate": "YYYY-MM-DDTHH:MM:SS",
              "endDate": "YYYY-MM-DDTHH:MM:SS",
              "im